In [10]:
import numpy as np
import pandas as pd
import re
import ssl
import nltk
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dinko\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dinko\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
netflix_dataset = pd.read_csv('./netflix_titles.csv')
netflix_dataset.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [12]:
netflix_dataset.groupby('type').count()

,show_id,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
type,,,,,,,,,,,
Movie,6131,6131,5943,5656,5691,6131,6131,6129,6128,6131,6131
TV Show,2676,2676,230,2326,2285,2666,2676,2674,2676,2676,2676


In [13]:
netflix_dataset.describe(include=["object", "bool"])

,show_id,type,title,director,cast,country,date_added,rating,duration,listed_in,description
count,8807,8807,8807,6173,7982,7976,8797,8803,8804,8807,8807
unique,8807,2,8807,4528,7692,748,1767,17,220,514,8775
top,s1,Movie,Dick Johnson Is Dead,Rajiv Chilaka,David Attenborough,United States,"January 1, 2020",TV-MA,1 Season,"Dramas, International Movies","Paranormal activity at a lush, abandoned prope..."
freq,1,6131,1,19,19,2818,109,3207,1793,362,4


In [14]:
netflix_dataset.isna().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [15]:
netflix_dataset = netflix_dataset.dropna(subset=['cast', 'country', 'rating'])
netflix_dataset['director'].fillna('Unkown', inplace = True)
netflix_dataset.isna().sum()

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      9
release_year    0
rating          0
duration        3
listed_in       0
description     0
dtype: int64

In [16]:
movies = netflix_dataset[netflix_dataset['type'] == 'Movie'].reset_index()
movies = movies.drop(['index', 'release_year', 'show_id', 'duration', 'date_added', 'description', 'type'], axis=1)
movies.head()

,title,director,cast,country,rating,listed_in
0,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",TV-MA,"Dramas, Independent Movies, International Movies"
1,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,PG-13,"Comedies, Dramas"
2,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic",TV-MA,"Dramas, International Movies"
3,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,TV-14,"Comedies, International Movies, Romantic Movies"
4,Grown Ups,Dennis Dugan,"Adam Sandler, Kevin James, Chris Rock, David S...",United States,PG-13,Comedies


In [17]:
tv_shows = netflix_dataset[netflix_dataset['type'] == 'TV Show'].reset_index()
tv_shows = tv_shows.drop(['index', 'release_year', 'show_id', 'duration', 'date_added', 'description', 'type'], axis=1)
tv_shows.head()

,title,director,cast,country,rating,listed_in
0,Blood & Water,Unkown,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,TV-MA,"International TV Shows, TV Dramas, TV Mysteries"
1,Kota Factory,Unkown,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,TV-MA,"International TV Shows, Romantic TV Shows, TV ..."
2,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,TV-14,"British TV Shows, Reality TV"
3,Dear White People,Unkown,"Logan Browning, Brandon P. Bell, DeRon Horton,...",United States,TV-MA,"TV Comedies, TV Dramas"
4,Falsa identidad,Unkown,"Luis Ernesto Franco, Camila Sodi, Sergio Goyri...",Mexico,TV-MA,"Crime TV Shows, Spanish-Language TV Shows, TV ..."


In [18]:
movie_genres = []

for i in movies['listed_in']:
    movie_genre = re.split(r', \s*', i)
    movie_genres.append(movie_genre)
    
temp_movie_genre_list = []
for sublist in movie_genres:
    for element in sublist:
        temp_movie_genre_list.append(element)
        
movie_genres_list = sorted(set(temp_movie_genre_list))

binary_movie_genres = [[0] * 0 for i in range(len(set(temp_movie_genre_list)))]

for i in movies['listed_in']:
    k = 0
    for j in movie_genres_list:
        if j.lower() in i.lower():
            binary_movie_genres[k].append(1.0)
        else:
            binary_movie_genres[k].append(0.0)
        k+=1
        
binary_movie_genres = pd.DataFrame(binary_movie_genres).transpose()
        
movie_countries = []

for i in movies['country']:
    movie_country = re.split(r', \s*', i)
    movie_countries.append(movie_country)
    
temp_movie_countries_list = []
for sublist in movie_countries:
    for element in sublist:
        temp_movie_countries_list.append(element)
        
movie_countries_list = sorted(set(temp_movie_countries_list))

binary_movie_countries = [[0] * 0 for i in range(len(set(temp_movie_countries_list)))]

for i in movies['country']:
    k = 0
    for j in movie_countries_list:
        if j.lower() in i.lower():
            binary_movie_countries[k].append(1.0)
        else:
            binary_movie_countries[k].append(0.0)
        k+=1
        
binary_movie_countries = pd.DataFrame(binary_movie_countries).transpose()

movie_ratings = []

for i in movies['rating']:
    movie_ratings.append(i)

movie_ratings_list = sorted(set(movie_ratings))

binary_movie_ratings = [[0] * 0 for i in range(len(set(movie_ratings_list)))]

for i in movies['rating']:
    k = 0
    for j in movie_ratings_list:
        if j.lower() in i.lower():
            binary_movie_ratings[k].append(1.0)
        else:
            binary_movie_ratings[k].append(0.0)
        k+=1
        
binary_movie_ratings = pd.DataFrame(binary_movie_ratings).transpose()

movie_directors = []

for i in movies['director']:
    if pd.notna(i):
        movie_director = re.split(r', \s*', i)
        movie_directors.append(movie_director)
    
temp_movie_directors_list = []
for sublist in movie_directors:
    for element in sublist:
        temp_movie_directors_list.append(element)
        
movie_directors_list = sorted(set(temp_movie_directors_list))

binary_movie_directors = [[0] * 0 for i in range(len(set(temp_movie_directors_list)))]

for i in movies['director']:
    k = 0
    for j in movie_directors_list:
        if i.lower() == 'unknown':
            binary_movie_directors[k].append(0.0)
        elif j.lower() in i.lower():
            binary_movie_directors[k].append(1.0)
        else:
            binary_movie_directors[k].append(0.0)
        k+=1
        
binary_movie_directors = pd.DataFrame(binary_movie_directors).transpose()

movie_actors = []

for i in movies['cast']:
    movie_actor = re.split(r', \s*', i)
    movie_actors.append(movie_actor)
    
temp_movie_actors_list = []
for sublist in movie_actors:
    for element in sublist:
        temp_movie_actors_list.append(element)
        
movie_actors_list = sorted(set(temp_movie_actors_list))

binary_movie_actors = [[0] * 0 for i in range(len(set(temp_movie_actors_list)))]

for i in movies['cast']:
    k = 0
    for j in movie_actors_list:
        if j.lower() in i.lower():
            binary_movie_actors[k].append(1.0)
        else:
            binary_movie_actors[k].append(0.0)
        k+=1
        
binary_movie_actors = pd.DataFrame(binary_movie_actors).transpose()
binary_movies = pd.concat([binary_movie_countries, binary_movie_actors,binary_movie_genres, binary_movie_directors], axis=1, ignore_index=True)
binary_movies

,0,1,2,3,4,5,6,7,8,9,...,29106,29107,29108,29109,29110,29111,29112,29113,29114,29115
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5275,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5276,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5277,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5278,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
tv_show_actors = []

for i in tv_shows['cast']:
    tv_show_actor = re.split(r', \s*', i)
    tv_show_actors.append(tv_show_actor)
    
    temp_tv_show_actors_list = []
    for sublist in tv_show_actors:
        for element in sublist: 
            temp_tv_show_actors_list.append(element)
        
tv_show_actors_list = sorted(set(temp_tv_show_actors_list))

binary_tv_show_actors = [[0] * 0 for i in range(len(set(temp_tv_show_actors_list)))]

for i in tv_shows['cast']:
    k = 0
    for j in tv_show_actors_list:
        if j.lower() in i.lower():
            binary_tv_show_actors[k].append(1.0)
        else:
            binary_tv_show_actors[k].append(0.0)
        k+=1

binary_tv_show_actors = pd.DataFrame(binary_tv_show_actors).transpose()

tv_show_countries = []
for i in tv_shows['country']:
    tv_show_country = re.split(r', \s*', i)
    tv_show_countries.append(tv_show_country)
    
temp_tv_show_countries_list = []
for sublist in tv_show_countries:
    for element in sublist: 
        temp_tv_show_countries_list.append(element)
        
tv_show_countries_list = sorted(set(temp_tv_show_countries_list))

binary_tv_show_countries = [[0] * 0 for i in range(len(set(temp_tv_show_countries_list)))]

for i in tv_shows['country']:
    k = 0
    for j in tv_show_countries_list:
        if j.lower() in i.lower():
            binary_tv_show_countries[k].append(1.0)
        else:
            binary_tv_show_countries[k].append(0.0)
        k+=1
        
binary_tv_show_countries = pd.DataFrame(binary_tv_show_countries).transpose()

tv_show_genres = []

for i in tv_shows['listed_in']:
    tv_show_genre = re.split(r', \s*', i)
    tv_show_genres.append(tv_show_genre)
    
temp_tv_show_genres_list = []
for sublist in tv_show_genres:
    for element in sublist:
        temp_tv_show_genres_list.append(element)
        
tv_show_genres_list = sorted(set(temp_tv_show_genres_list))

binary_tv_show_genres = [[0] * 0 for i in range(len(set(temp_tv_show_genres_list)))]

for i in tv_shows['listed_in']:
    k = 0
    for j in tv_show_genres_list:
        if j.lower() in i.lower(): 
            binary_tv_show_genres[k].append(1.0)
        else:
            binary_tv_show_genres[k].append(0.0)
        k+=1
        
binary_tv_show_genres = pd.DataFrame(binary_tv_show_genres).transpose()

tv_show_ratings = []

for i in tv_shows['rating']:
    tv_show_ratings.append(i)
    
tv_show_ratings_list = sorted(set(tv_show_ratings))

binary_tv_show_ratings = [[0] * 0 for i in range(len(set(tv_show_ratings_list)))]

for i in tv_shows['rating']:
    k = 0
    for j in tv_show_ratings_list:
        if j.lower() in i.lower():
            binary_tv_show_ratings[k].append(1.0)
        else:
            binary_tv_show_ratings[k].append(0.0)
        k+=1
        
binary_tv_show_ratings = pd.DataFrame(binary_tv_show_ratings).transpose()
binary_tv_shows = pd.concat([binary_tv_show_countries, binary_tv_show_actors, binary_tv_show_genres], axis=1, ignore_index=True)
binary_tv_shows

,0,1,2,3,4,5,6,7,8,9,...,13561,13562,13563,13564,13565,13566,13567,13568,13569,13570
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2020,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
def get_recommendations_using_multiple_feature_similarity(title_input):
    if title_input.lower() in movies['title'].str.lower().values:
        return(get_movie_recommendations_using_multiple_feature_similarity(title_input))
    elif title_input.lower() in tv_shows['title'].str.lower().values:
        return(get_tv_show_recommendations_using_multiple_feature_similarity(title_input))
    else:
        return('Title provided could not be found. Please try with another one')

In [21]:
def get_movie_recommendations_using_multiple_feature_similarity(title_input):
    movie_cosine_similarity_list = []
    binary_movie_list_one = []
    movie_index = movies[movies['title'].str.lower() == title_input.lower()].index.item()
    for i in binary_movies.iloc[movie_index]:
        binary_movie_list_one.append(i)
    point_one = np.array(binary_movie_list_one).reshape(1, -1)
    point_one = [val for sublist in point_one for val in sublist]
    for j in range(len(movies)):
        binary_movie_list_two = []
        for k in binary_movies.iloc[j]:
            binary_movie_list_two.append(k)
        point_two = np.array(binary_movie_list_two).reshape(1, -1)
        point_two = [val for sublist in point_two for val in sublist]
        dot_product = np.dot(point_one, point_two)
        norm_one = np.linalg.norm(point_one)
        norm_two = np.linalg.norm(point_two)
        cosine_similarity = dot_product / (norm_one * norm_two)
        movie_cosine_similarity_list.append(cosine_similarity)
    movies_temp_copy = movies.copy()
    movies_temp_copy['cosine_similarity'] = movie_cosine_similarity_list
    results = movies_temp_copy.sort_values('cosine_similarity', ascending=False)
    results = results[results['title'].str.lower() != title_input.lower()]
    top_ten_movie_results = results.head(10)
    return(top_ten_movie_results)

In [22]:
def get_tv_show_recommendations_using_multiple_feature_similarity(title_input):
    tv_show_cosine_similarity_list = []
    binary_tv_show_list = []
    tv_show_index = tv_shows[tv_shows['title'].str.lower() == title_input.lower()].index.item()
    for i in binary_tv_shows.iloc[tv_show_index]:
        binary_tv_show_list.append(i)
    point_one = np.array(binary_tv_show_list).reshape(1, -1)
    point_one = [val for sublist in point_one for val in sublist]
    for j in range(len(tv_shows)):
        binary_tv_show_list = []
        for k in binary_tv_shows.iloc[j]:
            binary_tv_show_list.append(k)
        point_two = np.array(binary_tv_show_list).reshape(1, -1)
        point_two = [val for sublist in point_two for val in sublist]
        dot_product = np.dot(point_one, point_two)
        norm_one = np.linalg.norm(point_one)
        norm_two = np.linalg.norm(point_two)
        cosine_similarity = dot_product / (norm_one * norm_two)
        tv_show_cosine_similarity_list.append(cosine_similarity)
    tv_shows_temp_copy = tv_shows.copy()
    tv_shows_temp_copy['cosine_similarity'] = tv_show_cosine_similarity_list
    results = tv_shows_temp_copy.sort_values('cosine_similarity', ascending=False)
    results = results[results['title'].str.lower() != title_input.lower()]
    top_ten_tv_show_results = results.head(10)
    return(top_ten_tv_show_results)

In [23]:
get_recommendations_using_multiple_feature_similarity('My Little Pony Equestria Girls: Forgotten Friendship')

,title,director,cast,country,rating,listed_in,cosine_similarity
2436,My Little Pony Equestria Girls: Rollercoaster ...,Ishi Rudell,"Ashleigh Ball, Andrea Libman, Rebecca Shoichet...","United States, Canada",TV-Y,Children & Family Movies,0.870388
3152,My Little Pony Equestria Girls: Legend of Ever...,Ishi Rudell,"Tara Strong, Rebecca Shoichet, Ashleigh Ball, ...","United States, Canada",TV-Y7,"Children & Family Movies, Comedies",0.804400
4384,My Little Pony Equestria Girls: Friendship Games,"Ishi Rudell, Jayson Thiessen","Tara Strong, Rebecca Shoichet, Ashleigh Ball, ...","United States, Canada",TV-Y,"Children & Family Movies, Comedies",0.804400
4385,My Little Pony Equestria Girls: Rainbow Rocks,"Jayson Thiessen, Ishi Rudell","Tara Strong, Ashleigh Ball, Andrea Libman, Tab...","United States, Canada",TV-Y,"Children & Family Movies, Comedies",0.710669
2289,My Little Pony Friendship Is Magic: Best Gift ...,"Denny Lu, Mike Myhre","Tara Strong, Andrea Libman, Ashleigh Ball, Cat...",United States,TV-Y,Children & Family Movies,0.622799
1114,Sugar High,Ariel Boles,Hunter March,United States,TV-G,Children & Family Movies,0.492366
4419,NOVA: Bird Brain,Unkown,Craig Sechler,United States,TV-G,"Children & Family Movies, Documentaries",0.455842
800,Dance Dreams: Hot Chocolate Nutcracker,Oliver Bokelberg,Debbie Allen,United States,TV-PG,"Children & Family Movies, Documentaries",0.455842
611,Animals on the Loose: A You vs. Wild Movie,Ben Simms,Bear Grylls,United States,TV-PG,Children & Family Movies,0.455842
2845,Ghost of the Mountains,Ben Wallis,Antoine Fuqua,United States,G,"Children & Family Movies, Documentaries",0.455842


In [24]:
get_recommendations_using_multiple_feature_similarity('13 Reasons Why')

,title,director,cast,country,rating,listed_in,cosine_similarity
941,MINDHUNTER,Unkown,"Jonathan Groff, Holt McCallany, Anna Torv, Cot...",United States,TV-MA,"Crime TV Shows, TV Dramas, TV Mysteries",0.451848
281,The Sinner,Unkown,"Jessica Biel, Bill Pullman, Christopher Abbott...",United States,TV-MA,"Crime TV Shows, TV Dramas, TV Mysteries",0.451848
1113,Unsolved,Unkown,"Josh Duhamel, Jimmi Simpson, Bokeem Woodbine",United States,TV-MA,"Crime TV Shows, TV Dramas",0.447214
934,13 Reasons Why: Beyond the Reasons,Unkown,"Dylan Minnette, Katherine Langford, Kate Walsh...",United States,TV-MA,"Crime TV Shows, Docuseries",0.447214
1666,Cold Case Files Classic,Unkown,Bill Kurtis,United States,TV-MA,"Crime TV Shows, Docuseries",0.422577
1932,Surviving R. Kelly Part II: The Reckoning,Unkown,R. Kelly,United States,TV-MA,"Crime TV Shows, Docuseries",0.422577
921,Surviving R. Kelly,Unkown,R. Kelly,United States,TV-MA,"Crime TV Shows, Docuseries",0.422577
1985,The Traffickers,Unkown,Nelufar Hedayat,United States,TV-MA,"Crime TV Shows, Docuseries",0.422577
1678,Deep Undercover Collection,Unkown,Dave B. Mitchell,United States,TV-MA,"Crime TV Shows, Docuseries",0.422577
143,The Assassination of Gianni Versace,Unkown,"Edgar Ramírez, Darren Criss, Ricky Martin, Pen...",United States,TV-MA,"Crime TV Shows, TV Dramas, TV Thrillers",0.404520


In [25]:
get_recommendations_using_multiple_feature_similarity('A Go! Go! Cory Carson Christmas')

,title,director,cast,country,rating,listed_in,cosine_similarity
1356,Go! Go! Cory Carson: The Chrissy,"Alex Woo, Stanley Moore","Alan C. Lim, Maisie Benson, Kerry Gudjohnsen, ...",United States,TV-Y,Children & Family Movies,0.788241
1114,Sugar High,Ariel Boles,Hunter March,United States,TV-G,Children & Family Movies,0.396059
868,Elf Pets: A Fox Cub’s Christmas Tale,Chanda Bell,"Leslie Bellair, Brad Hyland, Brody Rose, Carol...",United States,TV-Y,Children & Family Movies,0.383482
611,Animals on the Loose: A You vs. Wild Movie,Ben Simms,Bear Grylls,United States,TV-PG,Children & Family Movies,0.366679
800,Dance Dreams: Hot Chocolate Nutcracker,Oliver Bokelberg,Debbie Allen,United States,TV-PG,"Children & Family Movies, Documentaries",0.366679
2845,Ghost of the Mountains,Ben Wallis,Antoine Fuqua,United States,G,"Children & Family Movies, Documentaries",0.366679
4419,NOVA: Bird Brain,Unkown,Craig Sechler,United States,TV-G,"Children & Family Movies, Documentaries",0.366679
1773,LEGENDS OF THE HIDDEN TEMPLE,Joe Menendez,"Isabela Moner, Jet Jurgensmeyer, Colin Critchl...",United States,TV-PG,Children & Family Movies,0.365636
2245,Prince of Peoria: A Christmas Moose Miracle,Jon Rosenbaum,"Gavin Lewis, Theodore Barnes, Shelby Simmons, ...",United States,TV-G,"Children & Family Movies, Comedies",0.365636
821,Alien Xmas,Stephen Chiodo,"Keythe Farley, Dee Bradley Baker, Kaliayh Rham...",United States,TV-Y,Children & Family Movies,0.365636


In [26]:
movies_with_description = netflix_dataset[netflix_dataset['type'] == 'Movie'].reset_index()
movies_with_description = movies_with_description[['title', 'description']]
movies_with_description.head()

,title,description
0,Sankofa,"On a photo shoot in Ghana, an American model s..."
1,The Starling,A woman adjusting to life after a loss contend...
2,Je Suis Karl,After most of her family is murdered in a terr...
3,Jeans,When the father of the man she loves insists t...
4,Grown Ups,Mourning the loss of their beloved junior high...


In [27]:
tv_shows_with_description = netflix_dataset[netflix_dataset['type'] == 'TV Show'].reset_index()
tv_shows_with_description = tv_shows_with_description[['title', 'description']]
tv_shows_with_description.head()

,title,description
0,Blood & Water,"After crossing paths at a party, a Cape Town t..."
1,Kota Factory,In a city of coaching centers known to train I...
2,The Great British Baking Show,A talented batch of amateur bakers face off in...
3,Dear White People,Students of color navigate the daily slights a...
4,Falsa identidad,Strangers Diego and Isabel flee their home in ...


In [28]:
filtered_movie_descriptions = []

words_from_movies = []

for description_text in movies_with_description['description']:
    text_tokens = word_tokenize(description_text)
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words()]
    words_from_movies.append(tokens_without_sw)
    filtered = (" ").join(tokens_without_sw)
    filtered_movie_descriptions.append(filtered)

words_from_movies = [val for sublist in words_from_movies for val in sublist]
words_from_movies = sorted(set(words_from_movies))
movies_with_description['description_filtered'] = filtered_movie_descriptions
movies_with_description.head()

,title,description,description_filtered
0,Sankofa,"On a photo shoot in Ghana, an American model s...","on photo shoot ghana , american model slips ba..."
1,The Starling,A woman adjusting to life after a loss contend...,a woman adjusting life loss contends feisty bi...
2,Je Suis Karl,After most of her family is murdered in a terr...,"after family murdered terrorist bombing , youn..."
3,Jeans,When the father of the man she loves insists t...,when father loves insists twin sons marry twin...
4,Grown Ups,Mourning the loss of their beloved junior high...,mourning loss beloved junior high basketball c...


In [29]:
filtered_tv_show_descriptions = []
words_from_tv_shows = []

for description_text in tv_shows_with_description['description']:
    text_tokens = word_tokenize(description_text)
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words()]
    words_from_tv_shows.append(tokens_without_sw)
    filtered = (' ').join(tokens_without_sw)
    filtered_tv_show_descriptions.append(filtered)
    
words_from_tv_shows = [val for sublist in words_from_tv_shows for val in sublist]
words_from_tv_shows = sorted(set(words_from_tv_shows))
tv_shows_with_description['description_filtered'] = filtered_tv_show_descriptions
tv_shows_with_description.head()

,title,description,description_filtered
0,Blood & Water,"After crossing paths at a party, a Cape Town t...","after crossing paths party , cape town teen se..."
1,Kota Factory,In a city of coaching centers known to train I...,in city coaching centers known train india ’ f...
2,The Great British Baking Show,A talented batch of amateur bakers face off in...,a talented batch amateur bakers 10-week compet...
3,Dear White People,Students of color navigate the daily slights a...,students color navigate daily slights slippery...
4,Falsa identidad,Strangers Diego and Isabel flee their home in ...,strangers diego isabel flee home mexico preten...


In [30]:
binary_movies_with_description = [[0] * 0 for i in range(len(set(words_from_movies)))]

for filtered_desc in movies_with_description['description_filtered']:
    k = 0
    for word in words_from_movies:
        if word in filtered_desc:
            binary_movies_with_description[k].append(1.0)
        else:
            binary_movies_with_description[k].append(0.0)
        k+=1
        
binary_movies_with_description = pd.DataFrame(binary_movies_with_description).transpose()

In [31]:
binary_tv_shows_with_description = [[0] * 0 for i in range(len(set(words_from_tv_shows)))]

for filtered_desc in tv_shows_with_description['description_filtered']:
    k = 0
    for word in words_from_tv_shows:
        if word in filtered_desc:
            binary_tv_shows_with_description[k].append(1.0)
        else:
            binary_tv_shows_with_description[k].append(0.0)
        k+=1
    
binary_tv_shows_with_description = pd.DataFrame(binary_tv_shows_with_description).transpose()


In [32]:
def get_recommendations_using_description_similarity(title_input):
    if title_input.lower() in movies_with_description['title'].str.lower().values:
        return(get_movie_recommendations_using_description_similarity(title_input))
    elif title_input.lower() in tv_shows_with_description['title'].str.lower().values:
        return(get_tv_show_recommendations_using_description_similarity(title_input))
    else:
        return('Title provided could not be found. Please try with another one')

In [33]:
def get_movie_recommendations_using_description_similarity(title_input):
    movie_with_description_cosine_similarity_list = []
    binary_movie_with_description_list_one = []
    movie_index = movies_with_description[movies_with_description['title'].str.lower() == title_input.lower()].index.item()
    for i in binary_movies_with_description.iloc[movie_index]:
        binary_movie_with_description_list_one.append(i)
    point_one = np.array(binary_movie_with_description_list_one).reshape(1, -1)
    point_one = [val for sublist in point_one for val in sublist]    
    for j in range(len(movies_with_description)):
        binary_movie_with_description_list_two = []
        for k in binary_movies_with_description.iloc[j]:
            binary_movie_with_description_list_two.append(k)
        point_two = np.array(binary_movie_with_description_list_two).reshape(1, -1)
        point_two = [val for sublist in point_two for val in sublist]
        dot_product = np.dot(point_one, point_two)
        norm_one = np.linalg.norm(point_one)
        norm_two = np.linalg.norm(point_two)
        cosine_similarity = dot_product / (norm_one * norm_two)
        movie_with_description_cosine_similarity_list.append(cosine_similarity)
    movies_with_description_temp_copy = movies_with_description.copy()
    movies_with_description_temp_copy['cosine_similarity'] = movie_with_description_cosine_similarity_list
    results = movies_with_description_temp_copy.sort_values('cosine_similarity', ascending=False)
    results = results[results['title'].str.lower() != title_input.lower()]
    top_ten_movie_with_description_results = results.head(10)
    return(top_ten_movie_with_description_results)

In [34]:
def get_tv_show_recommendations_using_description_similarity(title_input):
    tv_show_with_description_cosine_similarity_list = []
    binary_tv_show_with_description_list_one = []
    tv_show_index = tv_shows_with_description[tv_shows_with_description['title'].str.lower() == title_input.lower()].index.item()
    for i in binary_tv_shows_with_description.iloc[tv_show_index]:
        binary_tv_show_with_description_list_one.append(i)
    point_one = np.array(binary_tv_show_with_description_list_one).reshape(1, -1)
    point_one = [val for sublist in point_one for val in sublist]
    for j in range(len(tv_shows_with_description)):
        binary_tv_show_with_description_list_two = []
        for k in binary_tv_shows_with_description.iloc[j]:
            binary_tv_show_with_description_list_two.append(k)
        point_two = np.array(binary_tv_show_with_description_list_two).reshape(1, -1)
        point_two = [val for sublist in point_two for val in sublist]
        dot_product = np.dot(point_one, point_two)
        norm_one = np.linalg.norm(point_one)
        norm_two = np.linalg.norm(point_two)
        cosine_similarity = dot_product / (norm_one * norm_two)
        tv_show_with_description_cosine_similarity_list.append(cosine_similarity)
    tv_shows_with_description_temp_copy = tv_shows_with_description.copy()
    tv_shows_with_description_temp_copy['cosine_similarity'] = tv_show_with_description_cosine_similarity_list
    results = tv_shows_with_description_temp_copy.sort_values('cosine_similarity', ascending=False)
    results = results[results['title'].str.lower() != title_input.lower()]
    top_ten_tv_show_with_description_results = results.head(10)
    return(top_ten_tv_show_with_description_results)

In [35]:
get_recommendations_using_description_similarity('My Little Pony Equestria Girls: Forgotten Friendship')

,title,description,description_filtered,cosine_similarity
4402,Natural Selection,"As the new kid, a shy high school senior finds...","as new kid , shy high school senior finds torm...",0.466782
3979,Hell and Back,"When best friends break a blood oath, one of t...","when best friends break blood oath , cast away...",0.465242
562,Just In Time,A meticulous clerk for a bookstore that has ju...,a meticulous clerk bookstore shuttered suddenl...,0.464420
89,C Kkompany,Three broke friends pose as an underworld gang...,three broke friends pose underworld gang extor...,0.455573
4747,Soul to Keep,Two siblings invite friends to their inherited...,"two siblings invite friends inherited , remote...",0.448775
406,Mystic River,"Haunted by a monstrous crime, three childhood ...","haunted monstrous crime , three childhood frie...",0.443203
3313,6-5=2,Six friends decide to undertake a grueling mou...,six friends decide undertake grueling mountain...,0.430730
1693,Spirit Riding Free: Spirit of Christmas,Lucky and friends must figure out how to get h...,lucky friends must figure get home time christ...,0.428174
5263,Yoko and His Friends,"Vik meets new friends in a new city, where the...","vik meets new friends new city , go exciting a...",0.426886
825,Break Ke Baad,Childhood friends Abhay and Aaliya experience ...,childhood friends abhay aaliya experience plen...,0.421329


In [36]:
get_recommendations_using_description_similarity('13 Reasons Why')

,title,description,description_filtered,cosine_similarity
857,"Love, Chunibyo & Other Delusions!",High schooler Yuta wants to leave his past as ...,high schooler yuta wants leave past delusional...,0.484329
638,Extracurricular,A model high school student who's steeped in a...,a model high school student 's steeped world s...,0.438202
465,Baby,"Fed up with their families and classmates, two...","fed families classmates , two teen girls wealt...",0.435989
855,CLANNAD,Maladjusted high school student Tomoya's life ...,maladjusted high school student tomoya 's life...,0.418854
1397,Beyblade Burst,"Middle schooler Valt Aoi, with his Beyblade Va...","middle schooler valt aoi , beyblade valtryek ,...",0.415139
526,Get Even,"In a secret act of skillful revenge, four priv...","in secret act skillful revenge , four private ...",0.408815
956,Let's Eat 2,Two classmates from elementary school reunite ...,two classmates elementary school reunite next-...,0.406579
1122,One Day at a Time,"In a reimagining of the TV classic, a newly si...","in reimagining tv classic , newly single latin...",0.400694
977,Angel Beats!,In a high school that's a way station to the a...,"in high school 's way station afterlife , teen...",0.400000
1503,Nightmare High,"After a new teacher arrives, students' dreams ...","after new teacher arrives , students ' dreams ...",0.398527


In [37]:
get_recommendations_using_description_similarity('A Go! Go! Cory Carson Christmas')

,title,description,description_filtered,cosine_similarity
375,Jungle Beat: The Movie,When a lost and lonely alien crash-lands on Ea...,"when lost lonely alien crash-lands earth , new...",0.460283
4575,Red Christmas,A family Christmas gathering at an isolated Au...,a family christmas gathering isolated australi...,0.448764
868,Elf Pets: A Fox Cub’s Christmas Tale,An elite team of elves – and their furry fox c...,an elite team elves – furry fox cub friends – ...,0.444944
1780,Christmas Survival,"With Christmas just around the corner, sisters...","with christmas around corner , sisters miranda...",0.444944
5073,The spiral,When an unbearable stranger arrives at a winte...,when unbearable stranger arrives winter mounta...,0.437073
2292,Puppy Star Christmas,A new pack of Pup Star pooches stumbles upon a...,a new pack pup star pooches stumbles upon riva...,0.433289
499,Irul,When a car breakdown forces a couple to seek s...,when car breakdown forces couple seek shelter ...,0.430588
3703,Dante's Peak,When a volcanologist comes to investigate the ...,when volcanologist comes investigate recent ru...,0.429601
3316,A Bad Moms Christmas,"Stressed-out moms Amy, Carla and Kiki are back...","stressed-out moms amy , carla kiki back , loom...",0.426006
3181,X: Past Is Present,When a middle-aged filmmaker meets an alluring...,when middle-aged filmmaker meets alluring stra...,0.422577
